# Evaluation and BacktestingThis notebook demonstrates how to evaluate forecasters using backtesting.

## Creating Forecast Task

## Simple Backtest

## Summarizing Results

## Expanding Window Split

## Sliding Window Split

## Model Comparison